# Identify working and data directory and site ID.
* The working directory is the root directory of the github.
* The data directory points to the google drive folder where the larger data files are stored.
* For the moment the site ID if the four-digit NEON site ID.

In [ ]:
working<-"~/neon_suna"
working_data<-"/Users/jhosen/gdrive/SUNA_Data/"
site_id<-"LIRO"
#data<-paste("~/gdrive/SUNA_Data/SUNA_NEON/",site_id,"/2019",sep="")
data<-paste(working_data,"/SUNA_NEON/LIRO_2020/",sep="")

save<-paste(working_data,"/corrected_files/",sep="")

# Initialize required libraries, functions and formatting utilities.

In [ ]:
library(dplyr)
library(streamMetabolizer)
library(splitstackshape)
library(ggplot2)
library(lubridate)
library(reshape2)
library(neonUtilities)
library(progress)
library(patchwork)
library(doBy)
library(stringr)
#library(box)

theme_ts_space<-theme_grey() +
		theme(
#		panel.grid.major = element_blank(),
#		panel.grid.minor = element_blank(),
		panel.background = element_rect(fill="white", colour="black", size=2),
#		legend.key       = element_blank(),
#		legend.text      = element_text(size=20.5),
#		legend.text      = element_blank(),
#		legend.title     = element_text(size=20.5),
		axis.text.x = element_text(size=22,colour="black",hjust=1,angle=45),
		axis.text.y = element_text(size=22,colour="black",vjust=.3),
		axis.title.x = element_text(size=22),
		axis.title.y = element_text(size=22,vjust=-1),
#		plot.title = element_text(hjust = 0.5,size=22,face="bold"),		
#		legend.position  = "left",
		legend.position  = "none",
		plot.margin = unit(c(1,1,1,1), "cm"),
		panel.border = element_rect(colour = "black", fill=NA, size=2)
		)





# Setting up the workspace
In the cell below indicate the site that will be analyzed here according to its NEON 4-letter code. A list of files for that site is then placed in the vector "sunas".

If you downloaded the repo into your home director, which is found by going to "~/", then you don"t have to change the box path. FYI: Box is a new package for R that allows you to define a set of functions without creating a new package.

In [ ]:

#setwd(paste("~/gdrive/SUNA_data/SUNA_NEON/",site_id,sep=""))
#sunas<-list.files(pattern="*.csv")
#getwd()
#options(box.path = "~/neon_suna/box/")
#box::use(plots)

setwd(data)
sunas<-list.files(pattern="*.CSV")

#suna_cal<-readRDS("/Users/jhosen/gdrive/fdom/SUNA Calibration pdfs/suna_cal_comb.rds")
#suna_cal<-read.csv("/Users/jhosen/gdrive/SUNA_Data/SUNA Calibration pdfs/suna_0839_20200312.csv")
#suna_cal_t<-as.data.frame(t(suna_cal))
#names(suna_cal_t)<-suna_cal_t[1,]

# Data import and processing loop.
This step loads each data file, formats the data for use, and concatenates files from an individual site.

This step could take a while.

In [ ]:
paste("/Users/jhosen/gdrive/SUNA_Data/SUNA_NEON/LIRO_2020/cal_files/",cal,"_abbr.csv",sep="")

In [ ]:
cal
paste("/Users/jhosen/gdrive/SUNA_Data/SUNA_NEON/LIRO_2020/cal_files/",cal,"_abbr.csv",sep="")
sunas[i]

In [ ]:
options(warn=-1)
compiled_suna<-data.frame()
sunas_length<-length(sunas)
for(i in 1:sunas_length){
#i<-1
print(i)


suna_calid<-read.csv(paste(sunas[i]),skip=0,header=FALSE)
cal<-str_extract(suna_calid,"[SNA]{3}[0-9]{4}[a-zA-Z]{1}")
cal<-cal[!is.na(cal)]
    
    
suna<-read.csv(paste(sunas[i]),skip=30,header=FALSE)

if(cal!="SNA0639C" & cal!="SNA0639D" & cal!="SNA0639F"){suna_cal<-read.csv(paste("/Users/jhosen/gdrive/SUNA_Data/SUNA_NEON/LIRO_2020/cal_files/",cal,"_abbr.csv",sep=""))}else{
    suna_cal<-read.csv("/Users/jhosen/gdrive/SUNA_Data/SUNA_NEON/LIRO_2020/CAL_files/SNA0639E_abbr.csv")}



    
    
suna_cal_t<-as.data.frame(t(suna_cal))
names(suna_cal_t)<-suna_cal_t[1,]

#suna_d0<-cSplit(suna,3, ",")
names(suna)<-c("suna_id","date","time","nitrate","nitrate_mgl","a254","a350","bromide_trace","spec_average_dark","dark_signal_average","int_time",suna_cal$wavelength_nm,"sensor_temp","spec_temp","lamp_temp","lamp_time","rel_hum","main_volt","lamp_volt","int_volt","main_current","fit_aux_1","fit_aux_2","fit_base_1","fit_base_2","fit_RMSE","CTD_Time","CTD_Salinity","CTD_Temp","CTD_Pressure","checksum")

    
#suna<-subset(suna,a254!=0)

suna<-subset(suna,a254!=0)
    
#year<-substr(suna$date, 1, 4)
if(grepl("D2019",sunas[i])==TRUE){
year<-"2019"
doy<-gsub(".CSV","",gsub("D2019","",paste(sunas[i]),fixed=TRUE),fixed=TRUE)
}
    
if(grepl("D2020",sunas[i])==TRUE){
year<-"2020"
doy<-gsub(".CSV","",gsub("D2020","",paste(sunas[i]),fixed=TRUE),fixed=TRUE)
}

day<-as.POSIXct(paste(as.Date(as.numeric(doy)-1,origin=paste(year,"-01-01",sep="")),"00:00:00"),tz="UTC")
#suna$day<-as.POSIXct("2019-05-10 00:00:00",tz="UTC")
suna$dtp<-day + (3600*as.numeric(suna$time))
#attr(suna$dtp,"tzone") <- "Etc/GMT+8"

    

offse<-c(t(suna_cal_t[3,]))
suna2<-(suna[,12:267]/suna$int_time)-suna$dark_signal_average
suna_d0_norm<-sweep(suna2,2,FUN="/",offse)
names(suna_d0_norm)<-paste("n_",names(suna_cal_t),sep="")

    

#print("test")
suna_d0_int<-data.frame()
for(j in 1:nrow(suna_d0_norm)){	
#    print(j)
    if(!is.na(suna_d0_norm[j,1])){
	flip<-as.data.frame(t(suna_d0_norm[j,]))
	names(flip)<-c("abs")
	flip$wl_nm<-gsub("n_","",row.names(flip),fixed=TRUE)
	flip_int<-as.data.frame(t(approx(flip$wl_nm,flip$abs,xout=seq(189,394,1),rule=2)$y))
	names(flip_int)<-paste("interp_",seq(189,394,1),sep="")
	suna_d0_int<-bind_rows(suna_d0_int,flip_int)
    }else{
        flip_int[1,]<-NA
       	suna_d0_int<-bind_rows(suna_d0_int,flip_int)
    }
}
suna_d<-bind_cols(suna,suna_d0_int)
#suna_d$date<-as.character(suna_d[,c("date")])
#suna_d$time<-as.character(suna_d[,c("time")])
suna_d2<-suna_d %>% mutate_if(is.numeric,as.character)
compiled_suna<-bind_rows(compiled_suna,suna_d2)
}
print("done")
options(warn=0)    
    
    
    


# Check Data and Save

In [ ]:
#head(compiled_suna)
#grepl("D2020",sunas[240])
#sunas[240]
#length(sunas)
#sunas[240]
#ncol(compiled_suna)
#nrow(compiled_suna)
#names(compiled_suna)<-gsub("inter_time","int_time",names(compiled_suna),fixed=TRUE)
#saveRDS(compiled_suna,"/Users/jhosen/gdrive/SUNA_Data/SUNA_NEON/LIRO_2020/LIRO_corrected/SUNA_LIRO_20210428_full.rds")
saveRDS(compiled_suna,paste(save,"/SUNA_Cor/","SUNA_LIRO_2020_20210701_full.rds",sep=""))
#compiled_suna<-readRDS("/Users/jhosen/gdrive/SUNA_Data/SUNA_NEON/LIRO_2020/LIRO_corrected/SUNA_LIRO_20210428_full.rds")


# Extracting the interpolated columns

In [ ]:
compiled_suna$cal<-"SNA1073"

suna_interp<-compiled_suna[,grepl("^inter",names(compiled_suna))] %>% mutate_if(is.character, ~as.numeric(.))
#head(suna_interp)

other_suna<-compiled_suna[,c("suna_id","date","time","nitrate","nitrate_mgl","a254","a350","bromide_trace","spec_average_dark","dark_signal_average","int_time","sensor_temp","spec_temp","lamp_temp","lamp_time","rel_hum","main_volt","lamp_volt","int_volt","main_current","fit_aux_1","fit_aux_2","fit_base_1","fit_base_2","fit_RMSE","CTD_Time","CTD_Salinity","CTD_Temp","CTD_Pressure","checksum","cal","dtp")]

suna_cols<-bind_cols(other_suna,suna_interp)
suna_cols$a254<-as.numeric(suna_cols$a254)
suna_cols$a350<-as.numeric(suna_cols$a350)
suna_cols$nitrate<-as.numeric(suna_cols$nitrate)
suna_cols$nitrate_mgl<-as.numeric(suna_cols$nitrate_mgl)
suna_cols$sensor_temp<-as.numeric(suna_cols$sensor_temp)


In [ ]:
range(suna_cols$a254)
range(suna_cols$interp_254)
suna_cols$interp_254
names(suna_cols)

In [ ]:
suna_cols<-subset(suna_cols,a350<10 & a254<10)
a350_mod<-lm(a350~log10(interp_350)+I(log10(interp_350)^2)+I(log10(interp_350)^3),suna_cols)
a254_mod<-lm(a254~log10(interp_254+0.6)+I(log10(interp_254+0.6)^2)+I(log10(interp_254+0.6)^3),suna_cols)



cor_cols<-grep("^interp_",names(suna_cols))


In [ ]:
ggplot(suna_cols,aes(interp_254,a254))+geom_point()

summary(a254_mod)
summary(resid(a254_mod))

summary(a350_mod)
summary(resid(a350_mod))

In [ ]:
suna_cols$interp_254[abs(resid(a254_mod))>0.05]<-NA
suna_cols$interp_350[abs(resid(a350_mod))>0.05]<-NA

a350_mod<-lm(a350~log10(interp_350)+I(log10(interp_350)^2)+I(log10(interp_350)^3),suna_cols)
a254_mod<-lm(a254~log10(interp_254)+I(log10(interp_254)^2)+I(log10(interp_254)^3),suna_cols)


summary(a350_mod)
summary(a254_mod)

In [ ]:
options(warn=1)


#compiled_suna2[,c(cor_cols)]<-compiled_suna2[,c(cor_cols)] %>% mutate_if(is.character,as.numeric)

for(i in 1:length(cor_cols)){
	prepdata<-data.frame(interp_350=suna_cols[,cor_cols[i]],interp_254=suna_cols[,cor_cols[i]])
	a350_pred<-predict(a350_mod,newdata=prepdata)
	a254_pred<-predict(a254_mod,newdata=prepdata)
	suna_cols[,c(paste(names(suna_cols)[cor_cols[i]],"_c350",sep=""))]<-a350_pred
	suna_cols[,c(paste(names(suna_cols)[cor_cols[i]],"_c254",sep=""))]<-a254_pred
}

options(warn=0)

# Reducing data down to 15 minutes intervals.
We want to average down each burst from the SUNA to a single value and mac sure that these values are snapped to 15 minutes intervals (e.g., rather than round up or down to 14 or 16).

In [ ]:
head(suna_cols)

In [ ]:
#rm(compiled_suna)
#rm(compiled_suna_1)
#rm(compiled_suna_2)

suna_cols$dtpr<-lubridate::round_date(suna_cols$dtp, "15 minutes")
suna_red<-summaryBy(.~dtpr,suna_cols,FUN=c(mean))



In [ ]:
#unique(as.Date(suna_red$dtpr))

# Downloading turbidity time series data from NEON.
Data product DP1.20288.001 for general water quality sonde data.

In [ ]:
LIRO_wqs<-loadByProduct(dpID="DP1.20288.001",site="LIRO",check.size=F)
saveRDS(LIRO_wqs,paste(data,"/LIRO_wqs.rds",sep=""))
LIRO_wqs<-readRDS(paste(data,"/LIRO_wqs.rds",sep=""))


In [ ]:
LIRO_wqsd<-LIRO_wqs$waq_instantaneous
LIRO_wqsd$dtp<-LIRO_wqsd$startDateTime
LIRO_wqk<-subset(LIRO_wqsd[,c("siteID","dtp","specificConductance","dissolvedOxygen","pH","chlorophyll","turbidity","fDOM")],!is.na(LIRO_wqsd$specificConductance))
#str(LIRO_wqk)

LIRO_wqk$dtpr<-lubridate::round_date(LIRO_wqk$dtp, "15 minutes")
LIRO_wq_red<-summaryBy(.~dtpr,LIRO_wqk,FUN=c(mean))
str(LIRO_wq_red)

In [ ]:
LIRO_sw<-merge(suna_red,LIRO_wq_red,by="dtpr",all.x=TRUE)


In [ ]:
head(LIRO_sw)
nrow(LIRO_sw)
save

In [ ]:
saveRDS(LIRO_sw,paste(save,"/SUNA_sonde_merge/","/LIRO_2020_SUNA_wq.rds",sep=""))

# import and merge NEON water quality grab sample data.

### Downloading grab sample data and saving as rds file in data folder.

In [ ]:


library(zoo)
LIRO_wqg<-loadByProduct(dpID="DP1.20093.001",site="LIRO",check.size=F)
saveRDS(LIRO_wqg,paste(data,"/LIRO_wqg.rds",sep=""))
LIRO_wqg<-readRDS(paste(data,"/LIRO_wqg.rds",sep=""))



### extracting lab analysis data

In [ ]:
LIRO_wqg_d<-as.data.frame(LIRO_wqg$swc_externalLabDataByAnalyte)

names(LIRO_wqg_d)

uv_abs<-subset(LIRO_wqg_d,analyte=="UV Absorbance (250 nm)"|analyte=="UV Absorbance (280 nm)")
date_cast<-dcast(uv_abs[,c("analyte","collectDate","analyteConcentration")],collectDate~analyte,value.var="analyteConcentration",mean)
date_cast$dtpr<-lubridate::round_date(date_cast$collectDate, "15 minutes")

names(date_cast)

In [ ]:

suna_grab<-merge(LIRO_sw,date_cast,by="dtpr")

#temp_nitrate_zoo<-zoo(nitrate$nitrate_umL,nitrate$dtp)
#temp_n<-na.approx(temp_nitrate_zoo,xout=suna_red$dtp,na.rm=FALSE)
#suna_grab[,c("UV Absorbance (250 nm)")]
names(suna_grab)<-gsub("UV Absorbance (250 nm)","uva_250_lab",names(suna_grab),fixed=TRUE)
names(suna_grab)<-gsub("UV Absorbance (280 nm)","uva_280_lab",names(suna_grab),fixed=TRUE)



In [ ]:
#subset(suna_red,as.Date(dtpr)==as.Date("2019-03-04"))
#as.Date(date_cast$dtpr)
#unique(as.Date(suna_red$dtpr))

#suna_grab
#LIRO_wqg_d

date_cast

# Exploring turbidity corrections

In [ ]:
#names(suna_grab)
#suna_grab<-subset(suna_grab,!is.na(uva_250_lab)&!is.na(uva_280_lab)&uva_250_lab<2)
suna_grab<-subset(suna_grab,!is.na(uva_250_lab)|!is.na(uva_280_lab))
suna_grab$turbidity.mean[suna_grab$turbidity.mean<=0]<-1
suna_grab[,c("dtpr","a254.mean","uva_250_lab","uva_280_lab","interp_254_c254.mean","turbidity.mean")]

suna_grab$interp_254_log<-log10(suna_grab$interp_254_c254.mean)
suna_grab$turb_log<-log10(suna_grab$turbidity.mean)
nrow(suna_grab)
suna_grab$uva_250_lab
#suna_grab$interp_254_tcorr<-predict(lm(uva_250_lab~interp_254_c254.mean*turbidity.mean,suna_grab))
#suna_grab$interp_254_tcorr_log<-predict(lm(uva_250_lab~interp_254_log*turbidity.mean,suna_grab))
#suna_grab$interp_254_tcorr_log_turb<-predict(lm(uva_250_lab~interp_254_log*turb_log,suna_grab))


saveRDS(suna_grab,paste(save,"/SUNA_grab_merge/","/LIRO_2020_SUNA_wqg.rds",sep=""))

In [ ]:
suna_grab<-subset(suna_grab,a254.mean<7 & a254.mean>0 & !is.na(turbidity.mean))

suna_grab$interp_254_log<-log10(suna_grab$interp_254_c254.mean)
suna_grab$turb_log<-log10(suna_grab$turbidity.mean)

#suna_grab$turb_log
summary(lm(uva_250_lab~a254.mean*turb_log,suna_grab))
#summary(lm(uva_250_lab~interp_254_c254.mean*turb_log,suna_grab))

#suna_grab$interp_254_tcorr<-predict(lm(uva_250_lab~interp_254_c254.mean*turbidity.mean,suna_grab))
#suna_grab$interp_254_tcorr_log<-predict(lm(uva_250_lab~interp_254_log*turbidity.mean,suna_grab))
#suna_grab$interp_254_tcorr_log_turb<-predict(lm(uva_250_lab~interp_254_log*turb_log,suna_grab))

suna_grab[,c("uva_250_lab","interp_254_c254.mean")]
ggplot(suna_grab,aes(a254.mean,uva_250_lab))+
#theme_ts_space+
#xlab("\nUV Absorbance at 250 nm (Laboratory)")+
#ylab("UV Absorbance at 250 nm (SUNA Uncorrected)\n")+
geom_point(size=6)



In [ ]:
suna_grab$interp_254_log<-log10(suna_grab$interp_254_c254.mean)
suna_grab$turb_log<-log10(suna_grab$turbidity.mean)


summary(lm(uva_250_lab~interp_254_log*turb_log,suna_grab))
summary(lm(uva_250_lab~interp_254_c254.mean*turb_log,suna_grab))

suna_grab$interp_254_tcorr<-predict(lm(uva_250_lab~interp_254_c254.mean*turbidity.mean,suna_grab))
suna_grab$interp_254_tcorr_log<-predict(lm(uva_250_lab~interp_254_log*turbidity.mean,suna_grab))
suna_grab$interp_254_tcorr_log_turb<-predict(lm(uva_250_lab~interp_254_log*turb_log,suna_grab))


ggplot(suna_grab,aes(uva_250_lab,interp_254_c254.mean,color=pH.mean))+
theme_ts_space+
xlab("\nUV Absorbance at 250 nm (Laboratory)")+
ylab("UV Absorbance at 250 nm (SUNA Uncorrected)\n")+
geom_point(size=6)

ggsave("~/neon_suna/plots/LIRO_250_uncorrected.pdf",width = 20, height = 20, units = "cm")



ggplot(suna_grab,aes(uva_250_lab,interp_254_tcorr_log,color=pH.mean))+
theme_ts_space+
geom_smooth(method="lm",color="grey20")+
xlab("\nUV Absorbance at 250 nm (Laboratory)")+
ylab("UV Absorbance at 250 nm (SUNA Corrected)\n")+
geom_point(size=6)+
ggtitle("Turbidity Corrected")

ggsave("~/neon_suna/plots/LIRO_250_corrected.pdf",width = 20, height = 20, units = "cm")



#summary(lm(uva_250_lab~interp_254_c254.mean*turbidity.mean+sensor_temp.mean,suna_grab))
#summary(lm(uva_280_lab~interp_280_c350.mean*turbidity.mean+sensor_temp.mean,suna_grab))
#summary(lm(uva_280_lab~interp_280_c254.mean*turbidity.mean+sensor_temp.mean,suna_grab))

In [ ]:
summary(lm(uva_250_lab~interp_254_c254.mean*turb_log,suna_grab))
summary(lm(uva_250_lab~interp_254_tcorr_log*turb_log,suna_grab))


In [ ]:
names(LIRO_sw)

# Apply turbidity correction based on lab samples.

In [ ]:
cor_cols<-grep("^interp_",names(LIRO_sw))

uva_250_turb_lm<-lm(uva_250_lab~interp_254_log*turbidity.mean,suna_grab)

for(i in 1:length(cor_cols)){
	prepdata<-data.frame(interp_254_log=log10(LIRO_sw[,cor_cols[i]]),turbidity.mean=LIRO_sw$turbidity.mean)
	turb_cor_pred<-predict(uva_250_turb_lm,newdata=prepdata)
	LIRO_sw[,c(paste(names(LIRO_sw)[cor_cols[i]],"_turb",sep=""))]<-turb_cor_pred
}



In [ ]:
head(LIRO_sw)

In [ ]:
LIRO_swt_0<-LIRO_sw[,grep("*_turb$",names(LIRO_sw))]
LIRO_swt<-bind_cols(LIRO_sw[,c("dtpr","nitrate.mean","nitrate_mgl.mean","a254.mean","a350.mean","sensor_temp.mean","specificConductance.mean","dissolvedOxygen.mean","pH.mean","chlorophyll.mean","turbidity.mean","fDOM.mean")],LIRO_swt_0)





# Adding PAR data.

In [ ]:
LIRO_par<-loadByProduct(dpID="DP1.20042.001",site="LIRO",check.size=F)
saveRDS(LIRO_par,paste(getwd(),"/LIRO_par.rds",sep=""))
LIRO_par<-readRDS(paste(getwd(),"/LIRO_par.rds",sep=""))




In [ ]:
#lubridate::round_date(LIRO_parts$dtp, "15 minutes")
#getwd()
#LIRO_swt$dtpr

In [ ]:

LIRO_parts<-LIRO_par$PARWS_5min
LIRO_parts$dtp<-LIRO_parts$startDateTime
#str(LIRO_wqk)



LIRO_parts$dtpr<-lubridate::round_date(LIRO_parts$dtp, "15 minutes")
LIRO_parts_red<-summaryBy(PARMean~dtpr,LIRO_parts,FUN=c(mean))
head(LIRO_parts_red)
#str(LIRO_wq_red)

LIRO_swtpar<-merge(LIRO_swt,LIRO_parts_red,by="dtpr",all.x=TRUE)



saveRDS(LIRO_swtpar,paste(save,"/SUNA_PAR/","/LIRO_SUNA_PAR.rds",sep=""))


In [ ]:

str(LIRO_swtpar$dtpr)

attr(LIRO_swtpar$dtpr,"tzone") <- "Etc/GMT+5"

str(LIRO_swtpar$dtpr)
LIRO_swtpar$date<-as.Date(LIRO_swtpar$dtpr,tz="Etc/GMT+5")

LIRO_dates<-unique(LIRO_swtpar$date)

for(i in 1:length(LIRO_dates)){
    
    
}

In [ ]:

#LIROd<-subset(LIRO_swtpar,date==as.Date("2019-06-10"))
LIROd<-LIRO_swtpar

LIROdk<-LIROd[,grep("*c254.mean_turb$",names(LIROd))]
LIROdk$dtpr<-LIROd$dtpr

LIROdm<-subset(melt(LIROdk,id.vars=c("dtpr")),!is.na(value))
LIROdm$wavelength<-gsub("interp_","",LIROdm$variable,fixed=TRUE)
LIROdm$wavelength<-as.numeric(gsub("_c254.mean_turb","",LIROdm$wavelength,fixed=TRUE))



str(LIROdm)

# Trying some ridge plots to look at wavelength changes over time.
So far not looLIRO super great.

In [ ]:
#library(ggridges)
#scales::rescale(height)
#LIROdm$dtprn<-scales::rescale(as.numeric(LIROdm$dtpr))
#LIROdm$abs<-scales::rescale(as.numeric(LIROdm$value))


#ggplot(subset(LIROdm,wavelength>=275&wavelength<=295),aes(x = wavelength, y = dtprn, group=dtprn,height = abs)) +
#  geom_ridgeline(fill="grey80",alpha=0.6)

#ggsave("~/neon_suna/plots/LIROdm_ridge.pdf",width = 20, height = 40, units = "cm")


# Calculating spectral slope 275-295nm on each time step.

In [ ]:
names(LIROd)
names(LIROdm)

In [ ]:
ssm_275_295<-subset(LIROdm,wavelength>=275&wavelength<=295)
dtps<-unique(ssm_275_295$dtpr)
pb <- progress_bar$new(
	format = "  downloading [:bar] :percent eta: :eta",
	total = length(dtps), clear = FALSE, width= 60)
s275295_comp<-data.frame()


for(i in 1:length(dtps)){
	ssm_now<-subset(ssm_275_295,dtpr==dtps[i])
	
	ssm_now$am1<-ssm_now$value*100*2.3025851
	ssm_now$lnam1<-log(ssm_now$am1)


	if(sum(!is.na(ssm_now$lnam1))>2){
		s275295<-lm(ssm_now$lnam1~ssm_now$wavelength)$coefficients[2]*-1
		}else{s275295<-NA}
		s275295_temp<-data.frame(dtpr=dtps[i],s275295=s275295)
		s275295_comp<-bind_rows(s275295_comp,s275295_temp)
#		pb$tick()
}	
	


In [ ]:
ggplot(subset(ssm_275_295,variable=="interp_275_c254.mean_turb"),aes(dtpr,value))+geom_point()


In [ ]:
ssm_275_295<-subset(LIROdm,wavelength>=275&wavelength<=295)
dtps<-unique(ssm_275_295$dtpr)
pb <- progress_bar$new(
	format = "  downloading [:bar] :percent eta: :eta",
	total = length(dtps), clear = FALSE, width= 60)
s275295_comp<-data.frame()



	ssm_now<-subset(ssm_275_295,dtpr==dtps[i])
	
	ssm_now$am1<-ssm_now$value*100*2.3025851
	ssm_now$lnam1<-log(ssm_now$am1)


	if(sum(!is.na(ssm_now$lnam1))>2){
		s275295<-lm(ssm_now$lnam1~ssm_now$wavelength)$coefficients[2]*-1
		}else{s275295<-NA}
		s275295_temp<-data.frame(dtpr=dtps[i],s275295=s275295)
		s275295_comp<-bind_rows(s275295_comp,s275295_temp)
#		pb$tick()


In [ ]:
nrow(s275295_comp)
nrow(LIRO_swtpar)
LIRO_swtp_ss<-merge(LIRO_swtpar,s275295_comp,by="dtpr",all.x=TRUE)
saveRDS(LIRO_swtp_ss,paste(data,"LIRO_swtp_ss.rds",sep=""))

In [ ]:
sdates<-unique(as.Date(LIRO_swtp_ss$dtpr,tz="Etc/GMT+5"))

i<-5
comp_df<-data.frame()
#print(i)
start<-as.POSIXct(paste(sdates[i]-1,"22:00",tz="Etc/GMT+5"))
finish<-as.POSIXct(paste(sdates[i]+1,"03:00",tz="Etc/GMT+5"))
LIROday<-subset(LIRO_swtp_ss,dtpr>=start & dtpr<=finish)
LIROday<-subset(LIROday,!is.na(PARMean.mean) & !is.na(s275295))
nrow(LIROday)
    LIROday$PAR_roll<-as.numeric(c("NA",rollmean(LIROday$PARMean.mean,k=3,align=c("center"),na.fill=TRUE),"NA"))
    LIROday$s275295_roll<-as.numeric(c("NA",rollmean(LIROday$s275295,k=3,align=c("center"),na.fill=TRUE),"NA"))



    LIRO_night<-subset(LIROday,PARMean.mean<200)
    night_s275295<-mean(LIRO_night$s275295,na.rm=TRUE)

    
    #LIROday$baseline<-predict(lm(s275295~dtpr,LIRO_night),LIROday)
    LIROday$baseline_roll<-predict(lm(s275295_roll~dtpr,LIRO_night),LIROday)    
    LIROday$s275295_bs<-LIROday$s275295_roll-LIROday$baseline_roll    
    
    LIRO_night_am<-subset(LIRO_night,hour(dtpr)<=12)
    night_s275295_am<-mean(LIRO_night_am$s275295,na.rm=TRUE)
    
    LIRO_night_pm<-subset(LIRO_night,hour(dtpr)>12)
    night_s275295_pm<-mean(LIRO_night_pm$s275295,na.rm=TRUE)
    
    LIRO_day<-subset(LIROday,PARMean.mean>=10)
    day_s275295<-mean(LIRO_day$s275295,na.rm=TRUE)

    day_s275295_base_sum<-sum(LIRO_day$s275295_bs,na.rm=TRUE)
    day_PAR_roll_base_sum<-sum(LIRO_day$PAR_roll,na.rm=TRUE)    

    int<-lm(s275295_bs~PAR_roll,LIRO_day)$coef[1]
    slope<-lm(s275295_bs~PAR_roll,LIRO_day)$coef[2]    
    
    LIRO_n_am_time<-nrow(LIRO_night_am)
    LIRO_n_pm_time<-nrow(LIRO_night_pm)
    LIRO_d_time<-nrow(LIRO_day)
    
    temp_df<-data.frame(site="LIRO",date=sdates[i],night_s275295=night_s275295,night_s275295_am=night_s275295_am,night_s275295_pm=night_s275295_pm,day_s275295=day_s275295,day_s275295_base_sum=day_s275295_base_sum,day_PAR_roll_base_sum=day_PAR_roll_base_sum,int=int,slope=slope,LIRO_n_am_time=LIRO_n_am_time,LIRO_n_pm_time=LIRO_n_pm_time,LIRO_d_time=LIRO_d_time)

# Extracting spectral daily slope ratio discrepancy.

In [ ]:
sdates<-unique(as.Date(LIRO_swtp_ss$dtpr,tz="Etc/GMT+5"))

#i<-30
comp_df<-data.frame()
for(i in 1:length(sdates)){
#print(i)
start<-as.POSIXct(paste(sdates[i]-1,"22:00",tz="Etc/GMT+5"))
finish<-as.POSIXct(paste(sdates[i]+1,"03:00",tz="Etc/GMT+5"))
LIROday<-subset(LIRO_swtp_ss,dtpr>=start & dtpr<=finish)
LIROday<-subset(LIROday,!is.na(PARMean.mean) & !is.na(s275295))

if(nrow(LIROday)>80){
    LIROday$PAR_roll<-as.numeric(c("NA",rollmean(LIROday$PARMean.mean,k=3,align=c("center"),na.fill=TRUE),"NA"))
    LIROday$s275295_roll<-as.numeric(c("NA",rollmean(LIROday$s275295,k=3,align=c("center"),na.fill=TRUE),"NA"))



    LIRO_night<-subset(LIROday,PARMean.mean<200)
    night_s275295<-mean(LIRO_night$s275295,na.rm=TRUE)
    
    #LIROday$baseline<-predict(lm(s275295~dtpr,LIRO_night),LIROday)
    LIROday$baseline_roll<-predict(lm(s275295_roll~dtpr,LIRO_night),LIROday)    
    LIROday$s275295_bs<-LIROday$s275295_roll-LIROday$baseline_roll    
    
    LIRO_night_am<-subset(LIRO_night,hour(dtpr)<=12)
    night_s275295_am<-mean(LIRO_night_am$s275295,na.rm=TRUE)
    
    LIRO_night_pm<-subset(LIRO_night,hour(dtpr)>12)
    night_s275295_pm<-mean(LIRO_night_pm$s275295,na.rm=TRUE)
    
    LIRO_day<-subset(LIROday,PARMean.mean>=10)
    day_s275295<-mean(LIRO_day$s275295,na.rm=TRUE)

    day_s275295_base_sum<-sum(LIRO_day$s275295_bs,na.rm=TRUE)
    day_PAR_roll_base_sum<-sum(LIRO_day$PAR_roll,na.rm=TRUE)    

    int<-lm(s275295_bs~PAR_roll,LIRO_day)$coef[1]
    slope<-lm(s275295_bs~PAR_roll,LIRO_day)$coef[2]    
    
    LIRO_n_am_time<-nrow(LIRO_night_am)
    LIRO_n_pm_time<-nrow(LIRO_night_pm)
    LIRO_d_time<-nrow(LIRO_day)
    
    temp_df<-data.frame(site="LIRO",date=sdates[i],night_s275295=night_s275295,night_s275295_am=night_s275295_am,night_s275295_pm=night_s275295_pm,day_s275295=day_s275295,day_s275295_base_sum=day_s275295_base_sum,day_PAR_roll_base_sum=day_PAR_roll_base_sum,int=int,slope=slope,LIRO_n_am_time=LIRO_n_am_time,LIRO_n_pm_time=LIRO_n_pm_time,LIRO_d_time=LIRO_d_time)
    comp_df<-bind_rows(comp_df,temp_df)
    }
}




In [ ]:
nrow(comp_df)
head(comp_df)

In [ ]:
    LIRO_day$s275295_bs
ggplot(comp_df,aes(date,day_s275295_base_sum))+
       geom_point(size=2)
ggplot(comp_df,aes(date,day_PAR_roll_base_sum))+
       geom_point(size=2)


ggplot(comp_df,aes(day_PAR_roll_base_sum,day_s275295_base_sum))+
geom_point()


saveRDS(comp_df,paste(data,"comp_df_LIRO.rds",sep=""))

In [ ]:
LIROday<-subset(LIRO_swtp_ss,as.Date(dtpr,tz="Etc/GMT+5")==as.Date("2019-06-11",tz="Etc/GMT+5"))
nrow(LIROday)

str(LIROday$s275295)



ggplot(LIROday,aes(dtpr,s275295))+
geom_point()


ggplot(LIROday,aes(dtpr,PARMean.mean))+
geom_point()

ggplot(LIROday,aes(dtpr,PARMean.mean))+
geom_point()

